In [62]:
# general imports
import numpy as np

# scipy imports
from scipy.fftpack import fft
from scipy.io import wavfile

# custom classes
%run classes/measure.py
%run classes/note.py

class Music:
    
    def __init__(self, 
                 time_signature=(4, 4),
                 tempo=60,
                 ver_number="0.00"):
            
        self.time_signature = time_signature
        self.tempo = tempo
        self.ver_number = ver_number # version number of decoder

    def read(self, input_path, is_wav_format=True):
        self.input_path = input_path
        if is_wav_format:
            self.sample_rate, self.raw = wavfile.read(input_path)
        self.chan1, self.chan2 = zip(*self.raw)
        
    def compile_music(self, window=1000, DIFF=15):
        self.measures = list()
        
        peaks = self.find_peaks(window, DIFF)
        notes = self.get_notes(peaks)
        notes = self.filter_notes(notes)
        for i, note in enumerate(notes):
            measure = Measure(i+1)
            measure.addNote(note)
            self.addMeasure(measure)
        return notes
    
    def get_notes(self, peaks, inspection_width=10000, use_chan1=True):
        notes = list()
        for peak in peaks:
            if use_chan1:
                inspection_zone = self.chan1[peak: peak+inspection_width]
                fft_data = np.abs(fft(inspection_zone))
                
                conversion_factor = self.sample_rate / len(fft_data)
                max_signal = max(fft_data)
                resonant_freqs = (-fft_data[:8000]).argsort()
                timestamp = peak / self.sample_rate

                for freq in resonant_freqs:
                    signal = fft_data[freq]
                    if signal < max_signal * 0.4:
                        break
                    note = Note(freq * conversion_factor, signal, timestamp)
                    notes.append(note.getInfo())
        notes = pd.DataFrame(notes, columns=["time", "id", "signal", "pitch", "given_pitch",
                                             "duration", "note", "octave", "alter"])
        return notes

    # ideally this is when dynamics will come in
    def filter_notes(self, notes):
        N = len(notes)
        to_delete = list()
        for i in range(1, N):
            if notes.iloc[i - 1].given_pitch == notes.iloc[i].given_pitch:
                to_delete.append(i)
        for index in list(reversed(to_delete)):
            notes.drop([index])
        return notes
        
    # Maybe there's a less computationally expensive way to find the start of notes instead of standard deviation?
    def find_peaks(self, window, DIFF):
        peaks = list()
        for i in range(window, len(self.chan1) - window, window):
            prev = self.chan1[i-window: i]
            curr = self.chan1[i: i+window]
            prev = np.average(np.abs(prev))
            curr = np.average(np.abs(curr))
            if curr > prev + DIFF:
                peaks.append(i)
        return peaks
        
    def addMeasure(self, measure):
        self.measures.append(measure)

In [71]:
music = Music()
music.read('sounds/wav/cello_pluck/expert/d3a3_copy6.wav')
notes = music.compile_music(window=500, DIFF=2000)
notes
# notes.groupby("time", "note")

,time,id,signal,pitch,given_pitch,duration,note,octave,alter
0,1.864583,46,3.652781e+07,216.0,220.00,4,A,3,0
1,1.864583,39,1.681620e+07,144.0,146.83,4,D,3,0
2,1.885417,46,3.521433e+07,216.0,220.00,4,A,3,0
3,1.885417,39,1.869047e+07,144.0,146.83,4,D,3,0
4,1.885417,58,1.533087e+07,432.0,440.00,4,A,4,0
